## Writeup Template
### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./output_images/image_correction.png "Undistorted"
[image2]: ./output_images/image_correction_samples.png "Road Transformed"
[image3]: ./output_images/thresholded_images.png "Binary Example"
[image3a]: ./output_images/mask_image.png "Mask"
[image3b]: ./output_images/clean_thresholded_images.png "Clean Binary Example"
[image4]: ./output_images/birdeye_view_images.png "Warp Example"
[image5]: ./output_images/fitted_lanelines.png "Fit Visual"
[image6]: ./output_images/marked_lane.png "Output"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the 5th code cell of the IPython notebook located in "./AdvancedLaneLines.ipynb". I broke this step to 2 small ultility functions:

* `calibrate_camera()`: This function calibrates the camera given the list of chessboard images and the grid dimensions of the chessboard. I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection. I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  

* `undistort_image()`: This function corrects the distortion of the given image, camera matrix and disortion coefficients. I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![alt text][image1]

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.
To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
![alt text][image2]
#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.
I used a combination of color and gradient thresholds to generate a binary image (thresholding steps at the 8th code cell of the IPython notebook located in "./AdvancedLaneLines.ipynb").  Here's an example of my output for this step.
![alt text][image3]

In other to refine the thresholded images, I applied a mask, which defined my region of interest, to the thresholded image.
![alt text][image3a]

The final result looks a lot cleaner as shown in the following figure:
![alt text][image3b]

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `warp_forward()` and `warp_backward()`, which appears in the 10th code cell of the IPython notebook. I chose the hardcode the source and destination points in the following manner:
```
#define the source coordinates
left_bottom = (220, 720)
right_bottom = (1280 - left_bottom[0], 720)
apex1 = (585, 460)
apex2 = (700, apex1[1])
birdeye_roi_src = np.array([[left_bottom, apex1, apex2, right_bottom,]], dtype=np.float32)

#define the destination coordinates
left_bottom = (300, 720)
right_bottom = (1280 - left_bottom[0], 720)
left_top = (left_bottom[0], 0)
right_top = (right_bottom[0], 0)
birdeye_roi_dst = np.array([[left_bottom, left_top, right_top, right_bottom,]], dtype=np.float32)

```
This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 220, 720      | 300, 720      | 
| 585, 460      | 300, 0        |
| 700, 460      | 980, 0        |
| 1060, 720     | 980, 720      |

With the 2 predefined sets of point, I calculated the 2 transforming matrices as ```perspectiveTransformMatrix```, which tranforms the source to the destination, and ```invPerspectiveTransformMatrix```, which tranforms the destination to the source.

The `warp_forward()` and `warp_backward()` function warp the input image (`inputImg`) from the normal viewing angle to the birdeye viewing angle and vice versa, respectively. 

I verified that my perspective transform was working as expected by drawing the `birdeye_roi_src` points (in red) onto a test images, and warped them to the birdeye view to verify that the lines appear parallel in the warped image.

![alt text][image4]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Then I did some other stuff and fit my lane lines with a 2nd order polynomial kinda like this:

![alt text][image5]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in lines # through # in my code in `my_other_file.py`

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines # through # in my code in `yet_another_file.py` in the function `map_lane()`.  Here is an example of my result on a test image:

![alt text][image6]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./detected_laneline_video.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  
